<a href="https://colab.research.google.com/github/eversonpereira/aulaML/blob/main/Aula_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Vamos começar importando as bibliotecas necessárias.

In [1]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, classification_report
import joblib

Carregaremos o dataset Iris

In [ ]:
iris = load_iris()
X = iris.data
y = iris.target



Dividiremos os dados em conjuntos de treinamento e teste



In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Instanciaremos o modelo KNN com k=3 (ajuste conforme necessário)

In [ ]:
knn = KNeighborsClassifier(n_neighbors=3)

Vamos treinar o modelo com os dados de treinamento

In [ ]:
knn.fit(X_train, y_train)

Avaliando o modelo com os dados de teste

In [ ]:
y_pred = knn.predict(X_test)

Vamos mostrar a matriz de confusão

In [ ]:
print("Matriz de Confusão:")
print(confusion_matrix(y_test, y_pred))

Vamos mostrar outras estatísticas

In [ ]:
print("\nEstatísticas de Desempenho:")
print(classification_report(y_test, y_pred))

Salvando o modelo treinado para uso futuro

In [ ]:
joblib.dump(knn, 'modelo_knn_iris.pkl')
print("Modelo salvo como 'modelo_knn_iris.pkl'")

Utilizando o modelo já treinado e realizando a classificação de um novo exemplo

In [ ]:
import numpy as np
import joblib

# Carregando o modelo treinado do arquivo
modelo_knn = joblib.load('modelo_knn_iris.pkl')

# Novo exemplo para classificar
novo_exemplo = np.array([[5.9, 3.0, 5.1, 1.8]])

# Fazendo a predição
predicao = modelo_knn.predict(novo_exemplo)

# Mostrando a classe prevista
print(f"Classe prevista: {predicao[0]}")
print(f"Nome da classe: {iris.target_names[predicao[0]]}")


Criando uma pseudo API onde receberá um post para rota predict e retornará a classificação. No corpo do POST deverá vim os atributos do exemplo a ser calssificado, por exemplo:

```
{
  "exemplo": [5.9, 3.0, 5.1, 1.8]
}

```
Esse código não funcionará no Colab pq não é possivel executar o app.run no colab.



In [ ]:
from flask import Flask, request, jsonify
import joblib
import numpy as np

app = Flask(__name__)

# Carrega o modelo treinado
modelo_knn = joblib.load('modelo_knn_iris.pkl')

@app.route('/predict', methods=['POST'])
def predict():
    data = request.get_json(force=True)
    # Assegura que os dados de entrada são convertidos para o formato correto
    medidas = np.array(data['exemplo']).reshape(1, -1)

    # Realiza a predição usando o modelo carregado
    predicao = modelo_knn.predict(medidas)

    # Retorna a classe prevista
    return jsonify({'classe': int(predicao[0])})

if __name__ == '__main__':
    app.run(debug=True)


Para funcionar no Colab vamos usar o ngrok para expor o servidor Flask, primeiro instalaremos as dependencias e depois utilizaremos o código abaixo:

In [ ]:
!pip install flask flask-ngrok


In [ ]:
from flask import Flask, request, jsonify
from flask_ngrok import run_with_ngrok
import joblib
import numpy as np

app = Flask(__name__)
run_with_ngrok(app)  # Inicia o ngrok quando o app é executado

# Carrega o modelo treinado
modelo_knn = joblib.load('modelo_knn_iris.pkl')

@app.route('/predict', methods=['POST'])
def predict():
    data = request.get_json(force=True)
    medidas = np.array(data['exemplo']).reshape(1, -1)
    predicao = modelo_knn.predict(medidas)
    return jsonify({'classe': int(predicao[0])})

if __name__ == '__main__':
    app.run()
